<a href="https://colab.research.google.com/github/PaoloBarba/ADM_HW2-Group18/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Algorithmic Methods of Data Mining**

#Academic year 2022–2023

# Homework 2 - Instagram Profiles & Posts

Authors: Barba Paolo, Soukaina Alaoui, Navid

#Import the packages and upload datasets


In [1]:
import pandas as pd
#import functions as fc
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline     
sns.set(color_codes=True)

In [2]:
from google.colab import drive 
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
locations =pd.read_csv('/content/drive/MyDrive/ADM_HW2/instagram_locations.csv',sep='\t')
profiles =pd.read_csv("/content/drive/MyDrive/ADM_HW2/instagram_profiles.csv",sep='\t')
posts =pd.read_csv("/content/drive/MyDrive/ADM_HW2/instagram_posts.csv",sep='\t',chunksize=4000000 )
#print(posts)


#Research question 1 Exploratory Data Analysis

Show the data

In [ ]:
locations.head()

In [ ]:
profiles.head()

In [ ]:
posts.head()

First of all we are going to first an info into the dataset to quickly understand the variables we are working with, get the numbers of rows and 
columns we have for each dataset

In [ ]:
print("Locations dataframe has " + str(locations.shape[0]) + " observations and "  + str(locations.shape[1]) + " variables")
print("Profiles dataframe has " + str(profiles.shape[0]) + " observations and "  + str(profiles.shape[1]) + " variables")
print("Posts dataframe has " + str(posts.shape[0]) + " observations and "  + str(posts.shape[1]) + " variables")

We are going one step further, having a better understanding of the statistical properties of these dataframes with a describe method. It's by having a statistical view of our data, we are going to be driving the process of analysis without the need of constantly looking at all rows we have.

In [ ]:
locations.describe()

In [ ]:
profiles.describe()

In [ ]:
posts.describe()

Now we can look at the columns of our datasets in order to understand which variables we are analyze.

In [ ]:
locations.info()

In [ ]:
profiles.info()

In [ ]:

posts.info()